In [2]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as  tradeapi
import time

import sys
sys.path.insert(0, '../')
import pandas as pd
from datetime import timedelta

In [3]:
from config import key_id, secret_key
from cross_signal import *

In [4]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [5]:
#The url at the end is what makes a paper trading bot
api = tradeapi.REST(key_id, secret_key)

In [6]:
account = api.get_account()
account

Account({   'account_blocked': False,
    'account_number': '010320JEWA',
    'buying_power': '0',
    'cash': '-4177.82',
    'created_at': '2020-03-20T17:46:19.948568Z',
    'currency': 'USD',
    'daytrade_count': 3,
    'daytrading_buying_power': '0',
    'equity': '3552.3',
    'id': '176ffd14-7e0e-4957-9ad5-98306425fa04',
    'initial_margin': '3865.06',
    'last_equity': '4161.4',
    'last_maintenance_margin': '2501.76',
    'long_market_value': '7730.12',
    'maintenance_margin': '2319.036',
    'multiplier': '2',
    'pattern_day_trader': False,
    'portfolio_value': '3552.3',
    'regt_buying_power': '0',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '18.09',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})

In [7]:
assetlist=api.list_positions()
for asset in assetlist:
    print(asset.symbol)

SE
AMD


In [8]:
assetlist

[Position({   'asset_class': 'us_equity',
     'asset_id': 'f9b42f64-98b9-4f8a-8ee2-e13d0a40f874',
     'avg_entry_price': '82.6099',
     'change_today': '-0.0733525733525734',
     'cost_basis': '8178.3801',
     'current_price': '77.06',
     'exchange': 'NYSE',
     'lastday_price': '83.16',
     'market_value': '7628.94',
     'qty': '99',
     'side': 'long',
     'symbol': 'SE',
     'unrealized_intraday_pl': '-603.9',
     'unrealized_intraday_plpc': '-0.0733525733525734',
     'unrealized_pl': '-549.4401',
     'unrealized_plpc': '-0.0671820205568582'}),
 Position({   'asset_class': 'us_equity',
     'asset_id': '03fb07bb-5db1-4077-8dea-5d711b272625',
     'avg_entry_price': '54.11995',
     'change_today': '-0.0490693739424704',
     'cost_basis': '108.2399',
     'current_price': '50.58',
     'exchange': 'NASDAQ',
     'lastday_price': '53.19',
     'market_value': '101.16',
     'qty': '2',
     'side': 'long',
     'symbol': 'AMD',
     'unrealized_intraday_pl': '-5.22',


In [9]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-27'

In [10]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['ema_high'] = tab.EMA(dataframe, timeperiod=4, price='high')
        dataframe['ema_close'] = tab.EMA(dataframe, timeperiod=4, price='close')
        dataframe['ema_low'] = tab.EMA(dataframe, timeperiod=4, price='low')
        
        dataframe['dema_close'] = tab.DEMA(dataframe, timeperiod=4, price='close')
        dataframe['sema_high'] = tab.EMA(dataframe, timeperiod=10, price='high')
        dataframe['sema_close'] = tab.EMA(dataframe, timeperiod=10, price='close')
        dataframe['sema_low'] = tab.EMA(dataframe, timeperiod=10, price='low')
        
        
        dataframe['chaikosc'] = ta.ADOSC(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        dataframe['mfi'] = ta.MFI(dataframe['high'],dataframe['low'],dataframe['close'], np.asarray(dataframe['volume'], dtype='float'), timeperiod=14)
        dataframe['ultosc'] = ta.ULTOSC(dataframe['high'],dataframe['low'],dataframe['close'])
        dataframe['obv'] = ta.OBV(dataframe['close'], np.asarray(dataframe['volume'], dtype='float'))
        
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['longminusdi'] = tab.MINUS_DI(dataframe, timeperiod=35)
        dataframe['longplusdi'] = tab.PLUS_DI(dataframe, timeperiod=35)
        
        
        dataframe['emaminussdi'] = ta.MINUS_DI(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['emaplusdi'] = ta.PLUS_DI(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['emaadx'] = ta.ADX(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        
        
        dataframe['srsi'] = tab.RSI(dataframe, timeperiod=20, price='close')
        #srsi used for finding buys at 71
        
        #dataframe['macd'],dataframe['macdsig'],dataframe['hist']
        
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)
        dataframe['shortadx'] = tab.ADX(dataframe, timeperiod=10)
        dataframe['srsi'] = tab.RSI(dataframe, timeperiod=3, price='close')
        dataframe["upperband"], dataframe["middleband"], dataframe["lowerband"] = ta.BBANDS(dataframe['close'],  nbdevup=1.5, nbdevdn=1.5)
        
        dataframe['ema_ultosc'] = ta.ULTOSC(dataframe['ema_high'],dataframe['ema_low'],dataframe['ema_close'])
        dataframe['ema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=3)
        dataframe['fema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=2)
        dataframe['sema_rsi'] = ta.RSI(dataframe['ema_close'], timeperiod=6)
        
        
        dataframe['beta'] = tab.BETA(dataframe)
        
        
        dataframe['adxcross1'] =crossed_above(dataframe['adx'],17)
        dataframe['adxcross2'] =crossed_above(dataframe['adx'],27)
        dataframe['adxcross3'] =crossed_above(dataframe['adx'],35)
        
        # required for graphing

        return dataframe

In [11]:
len(api.list_orders(status='closed'))

50

In [12]:
account = api.get_account()
print(account.status)


ACTIVE


In [153]:
#list of symbols to look at 
# DRI CLX 

In [154]:
symbol ='CDW'
#returned_data=api.polygon.historic_agg_v2('GOOG', 1, 'minute', _from='2020-01-10', to='2020-01-20').df
#returned_data =api.polygon.historic_agg_v2(symbol, 1, 'minute',_from='2020-04-27', to='2020-05-04').df
returned_data =api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df#
#returned_data=api.polygon.historic_agg_v2('UAL', 1, 'minute', _from='2020-05-13', to='2020-05-13').df
#returned_data=api.polygon.historic_agg_v2('XOM', 1, 'minute', _from='2020-05-11', to='2020-05-13').df


# Calculated trading indicators
data=populateindicators(returned_data)

In [155]:
asksizelist=[]
bidsizelist=[]
timestamplist=[]

In [156]:
api.polygon.last_quote('AMD')

Quote({   'askexchange': 19,
    'askprice': 49.78,
    'asksize': 3,
    'bidexchange': 19,
    'bidprice': 49.77,
    'bidsize': 6,
    'timestamp': 1590592192164})

In [167]:
import statistics as sts

In [674]:
quotes=api.polygon.historic_quotes_v2('LBTYA',date=startdate,reverse=True,limit=600)

In [730]:
quotesddd=api.polygon.historic_quotes_v2('LBTYA',date=startdate,reverse=True,limit=600).df

In [675]:
quotes2=api.polygon.historic_quotes_v2('SGEN',date=startdate,reverse=True,limit=1500)

In [714]:
quotes2[0].sip_timestamp

Timestamp('2020-05-27 14:36:22.595180271-0400', tz='America/New_York')

In [729]:
quotes2[30].sip_timestamp

Timestamp('2020-05-27 14:35:59.021088031-0400', tz='America/New_York')

In [701]:
asksizelist=[]
bidsizelist=[]
timestamplist=[]
askpricelist=[]
bidpricelist=[]

In [703]:
for quote in quotes:
    asksizelist.append(quote.ask_size)
    bidsizelist.append(quote.bid_size)
    bidpricelist.append(quote.bid_price)
    askpricelist.append(quote.ask_price)
    timestamplist.append(quote.sip_timestamp)

In [704]:
sum(bidsizelist)

3765

In [705]:
quotedf=pd.DataFrame({'bidpricelist':bidpricelist,'bidsizelist':bidsizelist,'askpricelist':askpricelist,'asksizelist':asksizelist,'timestamplist':timestamplist})

In [706]:
sum(asksizelist)

4082

In [707]:
sts.stdev(asksizelist)

3.5116769993554207

In [708]:
sts.median_low(bidpricelist)

21.06

In [709]:
sts.median_high(askpricelist)

21.07

In [710]:
quotes[-1].sip_timestamp

Timestamp('2020-05-27 14:34:22.580070803-0400', tz='America/New_York')

In [711]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = [x.upper() for x in symbols]
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)


In [712]:
quotedf=quotedf.reset_index()

In [732]:
# Use Dtale for graphs and inspection and find where the indicators indicate growth
import dtale

dtale.show(quotesddd, ignore_duplicate=True)


Executing shutdown due to inactivity...
Executing shutdown due to inactivity...


2020-05-27 18:09:23,477 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...
Executing shutdown...


2020-05-27 18:09:31,494 - INFO     - Executing shutdown...
